In [ ]:
!pip install matplotlib 
!pip install pandas
!pip install plotly
!pip install nbformat


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Constants
READ_SIZE_MB = 64/1024
INSERT_SIZE_MB = 1/1024
INSERT_BATCH_SIZE_MB = 100/1024

# Load + preprocess main log
df = pd.read_csv('logs/client_stats.log')
df['relative_time_s'] = (df['timestamp'] - df['timestamp'].min()) / 1000
df = df.sort_values(['client_id','op_type','timestamp'])
df['time_diff_s'] = df.groupby(['client_id','op_type'])['relative_time_s'].diff()
df['throughput'] = df['count']/df['time_diff_s']
df['throughput_mb_s'] = np.select(
    [df.op_type=='READ', df.op_type=='INSERT', df.op_type=='INSERT_BATCH'],
    [df.throughput*READ_SIZE_MB, df.throughput*INSERT_SIZE_MB, df.throughput*INSERT_BATCH_SIZE_MB],
    default=np.nan
)
df_th = df.dropna(subset=['throughput_mb_s'])

# Split by op_type
df_read = df_th[df_th.op_type=='READ'].copy()
df_insert = df_th[df_th.op_type=='INSERT']
df_insert_batch = df_th[df_th.op_type=='INSERT_BATCH']
df_queue = df[df.op_type=='QUEUE']
df_read_latency = df[df.op_type=='READ']
df_insert_latency = df[df.op_type=='INSERT']

# Cache metrics
df_read['user_cache_total'] = df_read.user_cache_hits + df_read.user_cache_misses
df_read['user_cache_hit_rate'] = np.where(df_read.user_cache_total>0,
                                          df_read.user_cache_hits/df_read.user_cache_total*100,
                                          np.nan)
df_cache = df_read[['relative_time_s','client_id','user_cache_usage']].dropna()
pivot_cache = df_cache.pivot_table(index='relative_time_s',
                                   columns='client_id',
                                   values='user_cache_usage',
                                   aggfunc='mean').fillna(0)

# Create subplots (13 rows)
fig = make_subplots(rows=13, cols=1, shared_xaxes=True, vertical_spacing=0.01,
                    subplot_titles=[
                        f"Per-Client READ Throughput for read size {READ_SIZE_MB*1024:.0f}KB",
                        "Per-Client WRITE Throughput (INSERT & INSERT_BATCH)",
                        "Combined READ & WRITE Throughput",
                        "Per-Client READ P10 Latency", "Per-Client READ P25 Latency",
                        "Per-Client READ P50 Latency", "Per-Client READ P99 Latency",
                        "Per-Client READ Max Latency", "Per-Client INSERT P99 Latency",
                        "Per-Client QUEUE P99 Latency", "Per-Client User Cache Hit Rate",
                        "Per-Client User Cache Usage (Stacked)",
                        "SSD Throughput Over Time"
                    ])

def add_group(df_group, row, ycol, name_fmt):
    for cid, grp in df_group.groupby('client_id'):
        fig.add_trace(go.Scatter(x=grp.relative_time_s, y=grp[ycol], name=name_fmt.format(cid)),
                      row=row, col=1)

# Populate subplots 1–12
add_group(df_read, 1, 'throughput_mb_s', "Client {}")
add_group(df_insert, 2, 'throughput_mb_s', "Client {} INSERT")
add_group(df_insert_batch, 2, 'throughput_mb_s', "Client {} INSERT_BATCH")
for (cid, op), grp in df_th.groupby(['client_id','op_type']):
    fig.add_trace(go.Scatter(x=grp.relative_time_s, y=grp.throughput_mb_s,
                             name=f"Client {cid}, {op}"), row=3, col=1)
for col_name, row_num in [('10p',4), ('25p',5), ('50p',6), ('99p',7), ('max',8)]:
    add_group(df_read_latency, row_num, col_name, "Client {}")
add_group(df_insert_latency, 9, '99p', "Client {}")
add_group(df_queue, 10, '99p', "Client {}")
add_group(df_read, 11, 'user_cache_hit_rate', "Client {}")
for cid in pivot_cache.columns:
    fig.add_trace(go.Scatter(x=pivot_cache.index, y=pivot_cache[cid]/1024**2,
                             name=f"Client {cid}", stackgroup="one"), row=12, col=1)

# === New SSD throughput subplot (row 13) — first 1/5th only ===
df_iostat = pd.read_csv("iostat_results.csv")
cutoff = len(df_iostat) // 5
time_seconds = np.arange(cutoff) + df['relative_time_s'].min()
fig.add_trace(go.Scatter(x=time_seconds, y=df_iostat["rMB/s"].iloc[:cutoff], name="SSD Read MB/s", mode="lines+markers"), row=13, col=1)
fig.add_trace(go.Scatter(x=time_seconds, y=df_iostat["wMB/s"].iloc[:cutoff], name="SSD Write MB/s", mode="lines+markers"), row=13, col=1)

# Axis labels
y_labels = ['Throughput (MB/s)']*3 + ['Latency (ms)']*7 + ['Hit Rate (%)','Cache Usage (MB)','MB/s']
for r,label in enumerate(y_labels, start=1):
    fig.update_yaxes(title_text=label, row=r, col=1)
fig.update_xaxes(title_text="Time (s)", row=13, col=1)
fig.update_yaxes(range=[0,400], row=5, col=1)

fig.update_layout(height=3900, width=1000, showlegend=False)
fig.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import json
import colorsys
from datetime import datetime
import plotly.express as px

# ---------------------------
# Main Log Processing & Plotting
# ---------------------------

# Constants
READ_SIZE_MB = 64/1024
INSERT_SIZE_MB = 1/1024
INSERT_BATCH_SIZE_MB = 100/1024

# Load + preprocess main log
df = pd.read_csv('logs/client_stats.log')
df['relative_time_s'] = (df['timestamp'] - df['timestamp'].min()) / 1000
df = df.sort_values(['client_id','op_type','timestamp'])
df['time_diff_s'] = df.groupby(['client_id','op_type'])['relative_time_s'].diff()
df['throughput'] = df['count'] / df['time_diff_s']
df['throughput_mb_s'] = np.select(
    [df.op_type=='READ', df.op_type=='INSERT', df.op_type=='INSERT_BATCH'],
    [df.throughput*READ_SIZE_MB, df.throughput*INSERT_SIZE_MB, df.throughput*INSERT_BATCH_SIZE_MB],
    default=np.nan
)
df_th = df.dropna(subset=['throughput_mb_s'])

# Split by op_type
df_read = df_th[df_th.op_type=='READ'].copy()
df_insert = df_th[df_th.op_type=='INSERT']
df_insert_batch = df_th[df_th.op_type=='INSERT_BATCH']
df_queue = df[df.op_type=='QUEUE']
df_read_latency = df[df.op_type=='READ']
df_insert_latency = df[df.op_type=='INSERT']

# Cache metrics
df_read['user_cache_total'] = df_read.user_cache_hits + df_read.user_cache_misses
df_read['user_cache_hit_rate'] = np.where(df_read.user_cache_total>0,
                                          df_read.user_cache_hits/df_read.user_cache_total*100,
                                          np.nan)
df_cache = df_read[['relative_time_s','client_id','user_cache_usage']].dropna()
pivot_cache = df_cache.pivot_table(index='relative_time_s',
                                   columns='client_id',
                                   values='user_cache_usage',
                                   aggfunc='mean').fillna(0)

# Create subplots (14 rows now; row 14 is for RocksDB events)
fig = make_subplots(rows=14, cols=1, shared_xaxes=True, vertical_spacing=0.01,
                    subplot_titles=[
                        f"Per-Client READ Throughput for read size {READ_SIZE_MB*1024:.0f}KB",
                        "Per-Client WRITE Throughput (INSERT & INSERT_BATCH)",
                        "Combined READ & WRITE Throughput",
                        "Per-Client READ P10 Latency", "Per-Client READ P25 Latency",
                        "Per-Client READ P50 Latency", "Per-Client READ P99 Latency",
                        "Per-Client READ Max Latency", "Per-Client INSERT P99 Latency",
                        "Per-Client QUEUE P99 Latency", "Per-Client User Cache Hit Rate",
                        "Per-Client User Cache Usage (Stacked)",
                        "SSD Throughput Over Time",
                        "RocksDB Events Over Time"  # New subplot row 14
                    ])

def add_group(df_group, row, ycol, name_fmt):
    for cid, grp in df_group.groupby('client_id'):
        fig.add_trace(go.Scatter(x=grp.relative_time_s, y=grp[ycol], name=name_fmt.format(cid)),
                      row=row, col=1)

# Populate subplots 1–12
add_group(df_read, 1, 'throughput_mb_s', "Client {}")
add_group(df_insert, 2, 'throughput_mb_s', "Client {} INSERT")
add_group(df_insert_batch, 2, 'throughput_mb_s', "Client {} INSERT_BATCH")
for (cid, op), grp in df_th.groupby(['client_id','op_type']):
    fig.add_trace(go.Scatter(x=grp.relative_time_s, y=grp.throughput_mb_s,
                             name=f"Client {cid}, {op}"), row=3, col=1)
for col_name, row_num in [('10p',4), ('25p',5), ('50p',6), ('99p',7), ('max',8)]:
    add_group(df_read_latency, row_num, col_name, "Client {}")
add_group(df_insert_latency, 9, '99p', "Client {}")
add_group(df_queue, 10, '99p', "Client {}")
add_group(df_read, 11, 'user_cache_hit_rate', "Client {}")
for cid in pivot_cache.columns:
    fig.add_trace(go.Scatter(x=pivot_cache.index, y=pivot_cache[cid]/1024**2,
                             name=f"Client {cid}", stackgroup="one"), row=12, col=1)

# SSD Throughput subplot (row 13) — first 1/5th only
df_iostat = pd.read_csv("iostat_results.csv")
# cutoff = len(df_iostat) // 5
cutoff = len(df_iostat)
time_seconds = np.arange(cutoff) + df['relative_time_s'].min()
fig.add_trace(go.Scatter(x=time_seconds, y=df_iostat["rMB/s"].iloc[:cutoff],
                         name="SSD Read MB/s", mode="lines+markers"), row=13, col=1)
fig.add_trace(go.Scatter(x=time_seconds, y=df_iostat["wMB/s"].iloc[:cutoff],
                         name="SSD Write MB/s", mode="lines+markers"), row=13, col=1)

# Axis labels
y_labels = ['Throughput (MB/s)']*3 + ['Latency (ms)']*7 + ['Hit Rate (%)','Cache Usage (MB)','MB/s', '']
for r, label in enumerate(y_labels, start=1):
    fig.update_yaxes(title_text=label, row=r, col=1)
fig.update_xaxes(title_text="Time (s)", row=14, col=1)
fig.update_yaxes(range=[0,400], row=5, col=1)

# ---------------------------
# RocksDB Events Plot (Row 14)
# ---------------------------
# For this plot, we assume the experiment start time is 0 (since main log times are already relative)
experiment_start_time = 0

# --- Helper functions for scalable client coloring ---

base_colors = {}

def get_client_base_color(client):
    """
    Returns a base color for the client. If the client does not have an assigned color,
    assign one from Plotly's 'Dark24' qualitative palette.
    """
    if client not in base_colors:
        palette = px.colors.qualitative.Dark24
        index = len(base_colors) % len(palette)
        base_colors[client] = palette[index]
    return base_colors[client]

def shade_color(base_color, level):
    """
    Returns a shade of the base color based on the level.
    Levels range from 1 (lightest) to 7 (darkest). Adjusts the lightness.
    """
    r, g, b = [int(base_color.lstrip('#')[i:i+2], 16)/255.0 for i in (0, 2, 4)]
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    new_l = 0.9 - (level - 1) * ((0.9 - 0.5) / 6)
    new_r, new_g, new_b = colorsys.hls_to_rgb(h, new_l, s)
    return '#%02x%02x%02x' % (int(new_r*255), int(new_g*255), int(new_b*255))

def get_client_color(client, level):
    """
    Returns a color for a given client and level by adjusting the client’s base color.
    """
    base = get_client_base_color(client)
    return shade_color(base, level)

# --- Parsing RocksDB Log and Adding Traces ---
# The log file path (adjust if needed)
log_file_path = '/mnt/rocksdb/ycsb-rocksdb-data/LOG'

# Compile regex patterns
flush_regex = re.compile(
    r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[/flush_job\.cc:\d+\] \[(.*?)\] \[JOB \d+\] Flush: (\d+) microseconds, \d+ cpu microseconds, (\d+) bytes'
)
l0_stall_pattern = re.compile(
    r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[WARN\] \[/column_family.cc:\d+\] \[([^,]+)\] Stalling writes because we have \d+ level-0 files rate (\d+)'
)
memtable_stall_pattern = re.compile(
    r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[WARN\] \[/column_family.cc:\d+\] \[([^,]+)\] Stalling writes because we have \d+ immutable memtables.*rate (\d+)'
)
pending_compaction_stall_pattern = re.compile(
    r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[WARN\] \[/column_family.cc:\d+\] \[([^,]+)\] Stalling writes because of estimated pending compaction bytes \d+ rate (\d+)'
)
memtable_stop_pattern = re.compile(
    r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[WARN\] \[/column_family.cc:\d+\] \[([^,]+)\] Stopping writes because we have \d+ immutable memtables.*'
)
compaction_regex = re.compile(r'.*EVENT_LOG_v1 (.*)$')

def timestamp_to_seconds(timestamp_str):
    ts = datetime.strptime(timestamp_str, '%Y/%m/%d-%H:%M:%S.%f')
    epoch = datetime(1970, 1, 1)
    return (ts - epoch).total_seconds()

def timestamp_to_micros(timestamp_str):
    dt = datetime.strptime(timestamp_str, '%Y/%m/%d-%H:%M:%S.%f')
    epoch = datetime(1970, 1, 1)
    return int((dt - epoch).total_seconds() * 1e6)

# Containers for events
l0_stalls = []
memtable_stalls = []
pending_compaction_stalls = []
compaction_data = {}  # keyed by client (cf_name)
flush_data = {}       # keyed by client
memtable_stops = []

with open(log_file_path, 'r') as log_file:
    for line in log_file:
        # L0 Stalls
        m = l0_stall_pattern.search(line)
        if m:
            timestamp_str, cf_name, rate = m.groups()
            ts = timestamp_to_micros(timestamp_str)
            l0_stalls.append((ts, int(rate)/1024/1024))
        # Memtable Stalls
        m = memtable_stall_pattern.search(line)
        if m:
            timestamp_str, cf_name, rate = m.groups()
            ts = timestamp_to_micros(timestamp_str)
            memtable_stalls.append((ts, int(rate)/1024/1024))
        # Memtable Stops
        m = memtable_stop_pattern.search(line)
        if m:
            timestamp_str, cf_name = m.groups()
            ts = timestamp_to_micros(timestamp_str)
            memtable_stops.append((ts, cf_name))
        # Pending Compaction Stalls
        m = pending_compaction_stall_pattern.search(line)
        if m:
            timestamp_str, cf_name, rate = m.groups()
            ts = timestamp_to_micros(timestamp_str)
            pending_compaction_stalls.append((ts, int(rate)/1024/1024))
        # Flush Events
        m = flush_regex.match(line)
        if m:
            timestamp_str, cf_name, flush_micro, flush_bytes = m.groups()
            start_sec = timestamp_to_seconds(timestamp_str) - int(flush_micro)/1e6
            rate = (int(flush_bytes) / int(flush_micro)) * 1e6 / (1024**2)
            flush_data.setdefault(cf_name, []).append((start_sec, rate, int(flush_micro)/1e6))
        # Compaction Events
        m = compaction_regex.match(line)
        if m:
            json_str = m.group(1)
            try:
                event = json.loads(json_str)
                if event.get('event') != 'compaction_finished':
                    continue
                end_sec = event['time_micros']/1e6
                start_sec = end_sec - event['compaction_time_micros']/1e6
                read_rate = event['read_rate']
                write_rate = event['write_rate']
                level = event['output_level']
                cf_name = event['cf_name']
                compaction_data.setdefault(cf_name, []).append((start_sec, end_sec, read_rate, write_rate, level))
            except Exception as e:
                print("Compaction json error:", e)

# Convert timestamps to relative times (seconds since experiment start)
l0_times = [ts/1e6 - experiment_start_time for ts, _ in l0_stalls]
l0_rates = [rate for _, rate in l0_stalls]

memtable_times = [ts/1e6 - experiment_start_time for ts, _ in memtable_stalls]
memtable_rates = [rate for _, rate in memtable_stalls]

pending_times = [ts/1e6 - experiment_start_time for ts, _ in pending_compaction_stalls]
pending_rates = [rate for _, rate in pending_compaction_stalls]

# Add stall events (as markers) on row 14
fig.add_trace(go.Scatter(
    x=l0_times, y=l0_rates, mode='markers',
    marker=dict(color='blue', size=6),
    name="L0 Stalls"
), row=14, col=1)
fig.add_trace(go.Scatter(
    x=memtable_times, y=memtable_rates, mode='markers',
    marker=dict(color='purple', size=6),
    name="Memtable Stalls"
), row=14, col=1)
fig.add_trace(go.Scatter(
    x=pending_times, y=pending_rates, mode='markers',
    marker=dict(color='orange', size=6),
    name="Pending Compaction Stalls"
), row=14, col=1)

# Add Flush events as horizontal lines per client
for cf_name, events in flush_data.items():
    base_color = get_client_base_color(cf_name)
    for i, (start_sec, rate, duration) in enumerate(events):
        dash_style = 'dash' if i == 0 else 'solid'
        fig.add_trace(go.Scatter(
            x=[start_sec - experiment_start_time, (start_sec + duration) - experiment_start_time],
            y=[rate, rate],
            mode='lines',
            line=dict(color=base_color, width=4, dash=dash_style),
            name=f"Client {cf_name} Flush",
            showlegend=True if i == 0 else False
        ), row=14, col=1)

# Add Compaction events as horizontal lines per event
for cf_name, events in compaction_data.items():
    for i, (start_sec, end_sec, read_rate, write_rate, level) in enumerate(events):
        color = get_client_color(cf_name, level)
        fig.add_trace(go.Scatter(
            x=[start_sec - experiment_start_time, end_sec - experiment_start_time],
            y=[write_rate, write_rate],
            mode='lines',
            line=dict(color=color, width=2),
            name=f"Client {cf_name} Compaction",
            showlegend=True if i == 0 else False
        ), row=14, col=1)

# Add Memtable Stops as vertical dashed lines (shapes)
# (Shapes are global; here we set y0=0 and y1 a reasonable max)
max_y = max(l0_rates + memtable_rates + pending_rates) if (l0_rates or memtable_rates or pending_rates) else 100
for ts, cf_name in memtable_stops:
    rel_time = ts/1e6 - experiment_start_time
    fig.add_shape(
        type="line",
        x0=rel_time, x1=rel_time,
        y0=0, y1=max_y,
        line=dict(color="brown", width=1, dash="dash"),
        opacity=0.5
    )
# Dummy trace for legend of memtable stops
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color="brown", width=1, dash="dash"),
    name="Memtable Stops"
), row=14, col=1)

# ---------------------------
# Final Layout Updates and Show Figure
# ---------------------------
fig.update_layout(height=3900, width=1000, showlegend=False)
fig.show()


In [13]:
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import json
import colorsys
from datetime import datetime
import plotly.express as px
import pickle

# ---------------------------
# Flags and Filenames
# ---------------------------

def plot_data(load_from_pickle, save_pickle, pickle_filename):
  # ---------------------------
  # Data Loading / Scraping Section
  # ---------------------------
  if load_from_pickle:
      with open(pickle_filename, 'rb') as f:
          data = pickle.load(f)
      # Extract dataframes and other variables from the pickled data
      df = data['df']
      df_th = data['df_th']
      df_read = data['df_read']
      df_insert = data['df_insert']
      df_insert_batch = data['df_insert_batch']
      df_queue = data['df_queue']
      df_read_latency = data['df_read_latency']
      df_insert_latency = data['df_insert_latency']
      pivot_cache = data['pivot_cache']
      df_iostat = data['df_iostat']
      l0_stalls = data['l0_stalls']
      memtable_stalls = data['memtable_stalls']
      pending_compaction_stalls = data['pending_compaction_stalls']
      memtable_stops = data['memtable_stops']
      flush_data = data['flush_data']
      compaction_data = data['compaction_data']
      experiment_start_time = data['experiment_start_time']
      time_seconds = data['time_seconds']
  else:
      # ---------------------------
      # Scrape and Process Log Data
      # ---------------------------
      # Constants
      READ_SIZE_MB = 64/1024
      INSERT_SIZE_MB = 1/1024
      INSERT_BATCH_SIZE_MB = 100/1024

      # Load and preprocess main log
      df = pd.read_csv('logs/client_stats.log')
      df['relative_time_s'] = (df['timestamp'] - df['timestamp'].min()) / 1000
      df = df.sort_values(['client_id', 'op_type', 'timestamp'])
      df['time_diff_s'] = df.groupby(['client_id', 'op_type'])['relative_time_s'].diff()
      df['throughput'] = df['count'] / df['time_diff_s']
      df['throughput_mb_s'] = np.select(
          [df.op_type=='READ', df.op_type=='INSERT', df.op_type=='INSERT_BATCH'],
          [df.throughput * READ_SIZE_MB, df.throughput * INSERT_SIZE_MB, df.throughput * INSERT_BATCH_SIZE_MB],
          default=np.nan
      )
      df_th = df.dropna(subset=['throughput_mb_s'])

      # Split by op_type
      df_read = df_th[df_th.op_type=='READ'].copy()
      df_insert = df_th[df_th.op_type=='INSERT']
      df_insert_batch = df_th[df_th.op_type=='INSERT_BATCH']
      df_queue = df[df.op_type=='QUEUE']
      df_read_latency = df[df.op_type=='READ']
      df_insert_latency = df[df.op_type=='INSERT']

      # Cache metrics
      df_read['user_cache_total'] = df_read.user_cache_hits + df_read.user_cache_misses
      df_read['user_cache_hit_rate'] = np.where(
          df_read.user_cache_total > 0,
          df_read.user_cache_hits / df_read.user_cache_total * 100,
          np.nan
      )
      df_cache = df_read[['relative_time_s', 'client_id', 'user_cache_usage']].dropna()
      pivot_cache = df_cache.pivot_table(index='relative_time_s',
                                        columns='client_id',
                                        values='user_cache_usage',
                                        aggfunc='mean').fillna(0)

      # Load iostat data
      df_iostat = pd.read_csv("iostat_results.csv")
      time_seconds = np.arange(len(df_iostat)) + df['relative_time_s'].min()

      # ---------------------------
      # Parse RocksDB Logs
      # ---------------------------
      experiment_start_time = 0

      # Regex patterns and helper functions
      flush_regex = re.compile(
          r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[/flush_job\.cc:\d+\] \[(.*?)\] \[JOB \d+\] Flush: (\d+) microseconds, \d+ cpu microseconds, (\d+) bytes'
      )
      l0_stall_pattern = re.compile(
          r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[WARN\] \[/column_family.cc:\d+\] \[([^,]+)\] Stalling writes because we have \d+ level-0 files rate (\d+)'
      )
      memtable_stall_pattern = re.compile(
          r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[WARN\] \[/column_family.cc:\d+\] \[([^,]+)\] Stalling writes because we have \d+ immutable memtables.*rate (\d+)'
      )
      pending_compaction_stall_pattern = re.compile(
          r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[WARN\] \[/column_family.cc:\d+\] \[([^,]+)\] Stalling writes because of estimated pending compaction bytes \d+ rate (\d+)'
      )
      memtable_stop_pattern = re.compile(
          r'(\d{4}/\d{2}/\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}) \d+ \[WARN\] \[/column_family.cc:\d+\] \[([^,]+)\] Stopping writes because we have \d+ immutable memtables.*'
      )
      compaction_regex = re.compile(r'.*EVENT_LOG_v1 (.*)$')

      def timestamp_to_seconds(timestamp_str):
          ts = datetime.strptime(timestamp_str, '%Y/%m/%d-%H:%M:%S.%f')
          epoch = datetime(1970, 1, 1)
          return (ts - epoch).total_seconds()

      def timestamp_to_micros(timestamp_str):
          dt = datetime.strptime(timestamp_str, '%Y/%m/%d-%H:%M:%S.%f')
          epoch = datetime(1970, 1, 1)
          return int((dt - epoch).total_seconds() * 1e6)

      # Containers for events
      l0_stalls = []
      memtable_stalls = []
      pending_compaction_stalls = []
      compaction_data = {}  # keyed by client (cf_name)
      flush_data = {}       # keyed by client
      memtable_stops = []

      log_file_path = '/mnt/rocksdb/ycsb-rocksdb-data/LOG'
      with open(log_file_path, 'r') as log_file:
          for line in log_file:
              m = l0_stall_pattern.search(line)
              if m:
                  timestamp_str, cf_name, rate = m.groups()
                  ts = timestamp_to_micros(timestamp_str)
                  l0_stalls.append((ts, int(rate)/1024/1024))
              m = memtable_stall_pattern.search(line)
              if m:
                  timestamp_str, cf_name, rate = m.groups()
                  ts = timestamp_to_micros(timestamp_str)
                  memtable_stalls.append((ts, int(rate)/1024/1024))
              m = memtable_stop_pattern.search(line)
              if m:
                  timestamp_str, cf_name = m.groups()
                  ts = timestamp_to_micros(timestamp_str)
                  memtable_stops.append((ts, cf_name))
              m = pending_compaction_stall_pattern.search(line)
              if m:
                  timestamp_str, cf_name, rate = m.groups()
                  ts = timestamp_to_micros(timestamp_str)
                  pending_compaction_stalls.append((ts, int(rate)/1024/1024))
              m = flush_regex.match(line)
              if m:
                  timestamp_str, cf_name, flush_micro, flush_bytes = m.groups()
                  start_sec = timestamp_to_seconds(timestamp_str) - int(flush_micro)/1e6
                  rate = (int(flush_bytes) / int(flush_micro)) * 1e6 / (1024**2)
                  flush_data.setdefault(cf_name, []).append((start_sec, rate, int(flush_micro)/1e6))
              m = compaction_regex.match(line)
              if m:
                  json_str = m.group(1)
                  try:
                      event = json.loads(json_str)
                      if event.get('event') != 'compaction_finished':
                          continue
                      end_sec = event['time_micros']/1e6
                      start_sec = end_sec - event['compaction_time_micros']/1e6
                      read_rate = event['read_rate']
                      write_rate = event['write_rate']
                      level = event['output_level']
                      cf_name = event['cf_name']
                      compaction_data.setdefault(cf_name, []).append((start_sec, end_sec, read_rate, write_rate, level))
                  except Exception as e:
                      print("Compaction json error:", e)

      # Adjust timestamps to be relative to experiment start time
      l0_stalls = [(ts, rate) for ts, rate in l0_stalls]
      memtable_stalls = [(ts, rate) for ts, rate in memtable_stalls]
      pending_compaction_stalls = [(ts, rate) for ts, rate in pending_compaction_stalls]
      # Convert to relative times (seconds) for plotting
      l0_times = [ts/1e6 - experiment_start_time for ts, _ in l0_stalls]
      l0_rates = [rate for _, rate in l0_stalls]
      memtable_times = [ts/1e6 - experiment_start_time for ts, _ in memtable_stalls]
      memtable_rates = [rate for _, rate in memtable_stalls]
      pending_times = [ts/1e6 - experiment_start_time for ts, _ in pending_compaction_stalls]
      pending_rates = [rate for _, rate in pending_compaction_stalls]

      # ---------------------------
      # Optionally Pickle the Data for Future Use
      # ---------------------------
      if save_pickle:
          data_to_pickle = {
              'df': df,
              'df_th': df_th,
              'df_read': df_read,
              'df_insert': df_insert,
              'df_insert_batch': df_insert_batch,
              'df_queue': df_queue,
              'df_read_latency': df_read_latency,
              'df_insert_latency': df_insert_latency,
              'pivot_cache': pivot_cache,
              'df_iostat': df_iostat,
              'l0_stalls': l0_stalls,
              'memtable_stalls': memtable_stalls,
              'pending_compaction_stalls': pending_compaction_stalls,
              'memtable_stops': memtable_stops,
              'flush_data': flush_data,
              'compaction_data': compaction_data,
              'experiment_start_time': experiment_start_time,
              'time_seconds': time_seconds
          }
          with open(pickle_filename, 'wb') as f:
              pickle.dump(data_to_pickle, f)
          print(f"Plot data pickled to {pickle_filename}")

  # ---------------------------
  # Plotting Section (Same for both data sources)
  # ---------------------------
  # Create subplots (14 rows; row 14 for RocksDB events)
  fig = make_subplots(rows=14, cols=1, shared_xaxes=True, vertical_spacing=0.01,
                      subplot_titles=[
                          f"Per-Client READ Throughput for read size {64:.0f}KB",
                          "Per-Client WRITE Throughput (INSERT & INSERT_BATCH)",
                          "Combined READ & WRITE Throughput",
                          "Per-Client READ P10 Latency", "Per-Client READ P25 Latency",
                          "Per-Client READ P50 Latency", "Per-Client READ P99 Latency",
                          "Per-Client READ Max Latency", "Per-Client INSERT P99 Latency",
                          "Per-Client QUEUE P99 Latency", "Per-Client User Cache Hit Rate",
                          "Per-Client User Cache Usage (Stacked)",
                          "SSD Throughput Over Time",
                          "RocksDB Events Over Time"
                      ])

  def add_group(df_group, row, ycol, name_fmt):
      for cid, grp in df_group.groupby('client_id'):
          fig.add_trace(go.Scatter(x=grp.relative_time_s, y=grp[ycol], name=name_fmt.format(cid)),
                        row=row, col=1)

  # Subplots 1–3 and 11–12 (using dataframes)
  add_group(df_read, 1, 'throughput_mb_s', "Client {}")
  add_group(df_insert, 2, 'throughput_mb_s', "Client {} INSERT")
  add_group(df_insert_batch, 2, 'throughput_mb_s', "Client {} INSERT_BATCH")
  for (cid, op), grp in df_th.groupby(['client_id', 'op_type']):
      fig.add_trace(go.Scatter(x=grp.relative_time_s, y=grp.throughput_mb_s,
                              name=f"Client {cid}, {op}"), row=3, col=1)
  for col_name, row_num in [('10p',4), ('25p',5), ('50p',6), ('99p',7), ('max',8)]:
      add_group(df_read_latency, row_num, col_name, "Client {}")
  add_group(df_insert_latency, 9, '99p', "Client {}")
  add_group(df_queue, 10, '99p', "Client {}")
  add_group(df_read, 11, 'user_cache_hit_rate', "Client {}")
  for cid in pivot_cache.columns:
      fig.add_trace(go.Scatter(x=pivot_cache.index, y=pivot_cache[cid]/1024**2,
                              name=f"Client {cid}", stackgroup="one"), row=12, col=1)

  # SSD Throughput subplot (row 13)
  fig.add_trace(go.Scatter(x=time_seconds, y=df_iostat["rMB/s"].iloc[:len(df_iostat)],
                          name="SSD Read MB/s", mode="lines+markers"), row=13, col=1)
  fig.add_trace(go.Scatter(x=time_seconds, y=df_iostat["wMB/s"].iloc[:len(df_iostat)],
                          name="SSD Write MB/s", mode="lines+markers"), row=13, col=1)

  # Axis labels and formatting
  y_labels = ['Throughput (MB/s)']*3 + ['Latency (ms)']*7 + ['Hit Rate (%)','Cache Usage (MB)','MB/s', '']
  for r, label in enumerate(y_labels, start=1):
      fig.update_yaxes(title_text=label, row=r, col=1)
  fig.update_xaxes(title_text="Time (s)", row=14, col=1)
  fig.update_yaxes(range=[0,400], row=5, col=1)

  # ---------------------------
  # RocksDB Events Plot (Row 14)
  # ---------------------------
  # Helper functions for client coloring
  base_colors = {}

  def get_client_base_color(client):
      if client not in base_colors:
          palette = px.colors.qualitative.Dark24
          index = len(base_colors) % len(palette)
          base_colors[client] = palette[index]
      return base_colors[client]

  def shade_color(base_color, level):
      r, g, b = [int(base_color.lstrip('#')[i:i+2], 16)/255.0 for i in (0, 2, 4)]
      h, l, s = colorsys.rgb_to_hls(r, g, b)
      new_l = 0.9 - (level - 1) * ((0.9 - 0.5) / 6)
      new_r, new_g, new_b = colorsys.hls_to_rgb(h, new_l, s)
      return '#%02x%02x%02x' % (int(new_r*255), int(new_g*255), int(new_b*255))

  def get_client_color(client, level):
      base = get_client_base_color(client)
      return shade_color(base, level)

  # L0, memtable, and pending compaction events as markers
  l0_times = [ts/1e6 - experiment_start_time for ts, _ in l0_stalls]
  l0_rates = [rate for _, rate in l0_stalls]
  memtable_times = [ts/1e6 - experiment_start_time for ts, _ in memtable_stalls]
  memtable_rates = [rate for _, rate in memtable_stalls]
  pending_times = [ts/1e6 - experiment_start_time for ts, _ in pending_compaction_stalls]
  pending_rates = [rate for _, rate in pending_compaction_stalls]

  fig.add_trace(go.Scatter(
      x=l0_times, y=l0_rates, mode='markers',
      marker=dict(color='blue', size=6),
      name="L0 Stalls"
  ), row=14, col=1)
  fig.add_trace(go.Scatter(
      x=memtable_times, y=memtable_rates, mode='markers',
      marker=dict(color='purple', size=6),
      name="Memtable Stalls"
  ), row=14, col=1)
  fig.add_trace(go.Scatter(
      x=pending_times, y=pending_rates, mode='markers',
      marker=dict(color='orange', size=6),
      name="Pending Compaction Stalls"
  ), row=14, col=1)

  # Flush events as horizontal lines
  for cf_name, events in flush_data.items():
      base_color = get_client_base_color(cf_name)
      for i, (start_sec, rate, duration) in enumerate(events):
          dash_style = 'dash' if i == 0 else 'solid'
          fig.add_trace(go.Scatter(
              x=[start_sec - experiment_start_time, (start_sec + duration) - experiment_start_time],
              y=[rate, rate],
              mode='lines',
              line=dict(color=base_color, width=4, dash=dash_style),
              name=f"Client {cf_name} Flush",
              showlegend=True if i == 0 else False
          ), row=14, col=1)

  # Compaction events as horizontal lines
  for cf_name, events in compaction_data.items():
      for i, (start_sec, end_sec, read_rate, write_rate, level) in enumerate(events):
          color = get_client_color(cf_name, level)
          fig.add_trace(go.Scatter(
              x=[start_sec - experiment_start_time, end_sec - experiment_start_time],
              y=[write_rate, write_rate],
              mode='lines',
              line=dict(color=color, width=2),
              name=f"Client {cf_name} Compaction",
              showlegend=True if i == 0 else False
          ), row=14, col=1)

  # Memtable stops as vertical dashed lines
  max_y = max(l0_rates + memtable_rates + pending_rates) if (l0_rates or memtable_rates or pending_rates) else 100
  for ts, cf_name in memtable_stops:
      rel_time = ts/1e6 - experiment_start_time
      fig.add_shape(
          type="line",
          x0=rel_time, x1=rel_time,
          y0=0, y1=max_y,
          line=dict(color="brown", width=1, dash="dash"),
          opacity=0.5
      )
  # Dummy trace for memtable stops legend
  fig.add_trace(go.Scatter(
      x=[None], y=[None],
      mode='lines',
      line=dict(color="brown", width=1, dash="dash"),
      name="Memtable Stops"
  ), row=14, col=1)

  fig.update_layout(height=3900, width=1000, showlegend=False)
  fig.show()

In [ ]:
LOAD_FROM_PICKLE = False   # Set to True to load data from pickle file instead of scraping logs
SAVE_PICKLE = False         # Set to True to pickle scraped data for future use

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
PICKLE_FILENAME = f'pickles/plot_data_{timestamp}.pkl'
# PICKLE_FILENAME = 'pickles/plot_data_20250402_192557.pkl'

plot_data(LOAD_FROM_PICKLE, SAVE_PICKLE, PICKLE_FILENAME)